In [1]:
from qiskit import *
import numpy as np 
from qiskit import QuantumCircuit, assemble, Aer
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from qiskit.providers.aer import QasmSimulator
from qiskit import Aer
from qiskit.circuit.library import LinearAmplitudeFunction
from qiskit_finance.circuit.library import LogNormalDistribution
from qiskit.providers.aer import *

In [5]:
def OR(qc,a,b,c):
    qc.x(a)
    qc.x(b)
    qc.x(c)
    qc.ccx(a,b,c)
    qc.x(a)
    qc.x(b)
qc=QuantumCircuit(18)
OR(qc,0,1,2)
qc.draw() 

┌───┐     ┌───┐
 q_0: ┤ X ├──■──┤ X ├
      ├───┤  │  ├───┤
 q_1: ┤ X ├──■──┤ X ├
      ├───┤┌─┴─┐└───┘
 q_2: ┤ X ├┤ X ├─────
      └───┘└───┘     
 q_3: ───────────────
                     
 q_4: ───────────────
                     
 q_5: ───────────────
                     
 q_6: ───────────────
                     
 q_7: ───────────────
                     
 q_8: ───────────────
                     
 q_9: ───────────────
                     
q_10: ───────────────
                     
q_11: ───────────────
                     
q_12: ───────────────
                     
q_13: ───────────────
                     
q_14: ───────────────
                     
q_15: ───────────────
                     
q_16: ───────────────
                     
q_17: ───────────────

In [78]:
def complement2(K,n):
    k=np.floor(K)
    c2=2**n-k   # sur 
    c2=int(c2)
    listec2=[]
    for i in range (n):
        listec2.append(c2%2)
        c2=c2//2
    return np.array(listec2)


def comparateur(qc,K,n,high,low):
    invK=(2**n-1)/(high-low)*(K-low)
    t=complement2(invK,n)
   
    for i in range(n):
        if ((i==0) or t[i]==0):
            qc.cx(i,n+i)
        else : 
            OR(qc,i,n+i-1,n+i)
    qc.cx(2*n-1,2*n)
    return 

def eq18(qc,n,K,c,low,high):
    comparateur(qc,K,n,high,low)
    invK=(2**n-1)/(high-low)*(K-low)
    theta = 2*c/(2**n-1-invK)
    qc.ry(2*(np.pi/4-c),2*n+1)
    qc.cry(-2*invK*theta,2*n,2*n+1)
    for k in range(n):
        qc.ucry([0,0,0,2*theta*(2**k)],[k,2*n],[2*n+1])
    return


In [84]:
n=3
c = 0.25
K=1.896


num_uncertainty_qubits = n
# parameters for considered random distribution
S = 2.0 # initial spot price
vol = 0.4 # volatility of 40%
r = 0.05 # annual interest rate of 4%
T = 40 / 365 # 40 days to maturity
# resulting parameters for log-normal distribution
mu = ((r - 0.5 * vol**2) * T + np.log(S))
sigma = vol * np.sqrt(T)
mean = np.exp(mu + sigma**2/2)
variance = (np.exp(sigma**2) - 1) * np.exp(2*mu + sigma**2)
stddev = np.sqrt(variance)
# lowest and highest value considered for the spot price; in between, an equidistant discretization is considered.
low  = np.maximum(0, mean - 3*stddev)
high = mean + 3*stddev
# construct A operator for QAE for the payoff function by
# composing the uncertainty model and the objective
uncertainty_model = LogNormalDistribution(num_uncertainty_qubits, mu=mu, sigma=sigma**2, bounds=(low, high))


essais=100000

qc = QuantumCircuit(2*n+2,1)
qc.append(uncertainty_model, range(n))

eq18(qc,n,K,c,low,high)
qc.measure(2*n+1,0)


simulator=QasmSimulator()
job=simulator.run(transpile(qc,simulator),shots=essais)
counts=job.result().get_counts(transpile(qc,simulator))
print(counts)
P1=counts.int_outcomes().get(1)/essais

print((P1-1/2+c)*(2**n-1-(2**n-1)/(high-low)*(K-low))/(2*c))


{'1': 32450, '0': 67550}
0.5962359411535061


In [74]:
#essais
n=3
c = 0.25
K= low+(high-low)/(2**n-1)



essais=10000

qc = QuantumCircuit(2*n+2,1)
qc.h(range(n))

comparateur(qc,K,n,high,low)
print(qc.draw())
qc.measure(2*n,0)


simulator=QasmSimulator()
job=simulator.run(transpile(qc,simulator),shots=essais)
counts=job.result().get_counts(transpile(qc,simulator))
print(counts)
print(low,high)

     ┌───┐                              
q_0: ┤ H ├──■───────────────────────────
     ├───┤  │  ┌───┐     ┌───┐          
q_1: ┤ H ├──┼──┤ X ├──■──┤ X ├──────────
     ├───┤  │  ├───┤  │  └───┘     ┌───┐
q_2: ┤ H ├──┼──┤ X ├──┼─────────■──┤ X ├
     └───┘┌─┴─┐├───┤  │  ┌───┐  │  └───┘
q_3: ─────┤ X ├┤ X ├──■──┤ X ├──┼───────
     ┌───┐└───┘└───┘┌─┴─┐├───┤  │  ┌───┐
q_4: ┤ X ├──────────┤ X ├┤ X ├──■──┤ X ├
     ├───┤          └───┘└───┘┌─┴─┐└───┘
q_5: ┤ X ├────────────────────┤ X ├──■──
     └───┘                    └───┘┌─┴─┐
q_6: ──────────────────────────────┤ X ├
                                   └───┘
q_7: ───────────────────────────────────
                                        
c: 1/═══════════════════════════════════
                                        
{'1': 8752, '0': 1248}
1.208607238741425 2.813370728095958
